In [1]:
try
    using Gmsh: gmsh
catch
    using gmsh
end

using GR 

using LinearAlgebra 
using SparseArrays 
using StaticArrays
using StaticRanges

using BenchmarkTools

using Test 

using Plots 

In [2]:
struct Point
    x::Float64   # x coordinates
    y::Float64   # y coordinates 
    z::Float64   # z coordinates
end

struct Element
    p1::Point
    p2::Point
    p3::Point
    p4::Point
    e1::Int64
    e2::Int64
    e3::Int64
    e4::Int64
    vol::Float64
end

struct Mesh
    nnodes::Int64
    nelements::Int64 
    # specify one-dimensional array of elements as an array of structs. 
    # we worry about using structArray (if as all) later. 
    Elements::Array{Element,1}
    bndNodeIds::Vector{Int64}
    dofPerElement::Int64       
end 

In [3]:
function volume_tetrahedron(x1::Float64,x2::Float64,x3::Float64,x4::Float64,y1::Float64,y2::Float64,y3::Float64,y4::Float64,z1::Float64,z2::Float64,z3::Float64,z4::Float64)::Float64
    mat = [[x1 y1 z1 1]; [x2 y2 z2 1]; [x3 y3 z3 1]; [x4 y4 z4 1]];
    volume = abs(det(mat))/6;
    return volume
end

volume_tetrahedron (generic function with 1 method)

In [4]:
# read elements from mesh file 
function meshFromGmsh(meshFile)    
    #the function takes the name of the file as input
    dofPerElement = 16; # DoF per element: 3 translations (along x, y, z directions) and temperature
    
    gmsh.initialize()
    gmsh.open(meshFile)
    
    node_ids, node_coord, _ = gmsh.model.mesh.getNodes()
    nnodes = length(node_ids)

    #..sort the node coordinates by ID, such that Node one sits at row 1
    tosort = [node_ids node_coord[1:3:end] node_coord[2:3:end] node_coord[3:3:end]]; 
    sorted = sortslices(tosort , dims = 1);

    node_ids = sorted[:,1]
    xnode = sorted[:,2]
    ynode = sorted[:,3]
    znode = sorted[:,4]

    #get the mesh elements
    element_types, element_ids, element_connectivity = gmsh.model.mesh.getElements(3)
    nelements = length(element_ids[1])
    #boundary elements
    bnd_types, bnd_el_ids, bnd_element_connectivity = gmsh.model.mesh.getElements(2)
    
    bnd_node_ids = [];
    #print(bnd_el_ids)
    for i in 1:length(bnd_el_ids[1])
        #retrieve global numbering of the local nodes of the current BOUNDARY element
        bnd_node1_id = bnd_element_connectivity[1][3*(i-1)+1]
        bnd_node2_id = bnd_element_connectivity[1][3*(i-1)+2]
        bnd_node3_id = bnd_element_connectivity[1][3*(i-1)+3]
        if !(bnd_node1_id in bnd_node_ids)
            push!(bnd_node_ids, bnd_node1_id)
        end
        if !(bnd_node2_id in bnd_node_ids)
            push!(bnd_node_ids, bnd_node2_id)
        end
        if !(bnd_node3_id in bnd_node_ids)
            push!(bnd_node_ids, bnd_node3_id)
        end
        
    end
    bnd_node_ids = sort(bnd_node_ids)
    
    Elements = Array{Element,1}(undef,nelements);
    total_volume = 0;
    for i in 1:nelements
        
        #....retrieve global numbering of the local nodes of the current element
        e1 = element_connectivity[1][4*(i-1)+1]
        e2 = element_connectivity[1][4*(i-1)+2]
        e3 = element_connectivity[1][4*(i-1)+3]
        e4 = element_connectivity[1][4*(i-1)+4]
        
        #....retrieve the x and y coordinates of the local nodes of the current element
        p1 = Point(sorted[e1,2], sorted[e1,3], sorted[e1,4])
        p2 = Point(sorted[e2,2], sorted[e2,3], sorted[e2,4])
        p3 = Point(sorted[e3,2], sorted[e3,3], sorted[e3,4])
        p4 = Point(sorted[e4,2], sorted[e4,3], sorted[e4,4])
        #calculate volume of the element and add it to the total volume
        vol = volume_tetrahedron(p1.x, p2.x, p3.x, p4.x, p1.y, p2.y, p3.y, p4.y, p1.z, p2.z, p3.z, p4.z)
        #total_volume = total_volume + vol;
        #save nodes coordinates, nodes IDs and the volume of one element
        Elements[i] = Element(p1, p2, p3, p4, e1, e2, e3, e4, vol)
    end
    #print(total_volume) #using cube_3D.msh the volume should be 1
    mesh = Mesh(nnodes, nelements, Elements, bnd_node_ids, dofPerElement)
    gmsh.finalize()
    
    return mesh
    
end

#..read nodes from mesh file (useful for post-processing)
function nodesFromGmsh(meshFile)
    
    #..Initialize GMSH
    gmsh.initialize()
    
    #..Read mesh from file
    gmsh.open(meshFile)

    #..Get the mesh nodes
    #..Observe that although the mesh is two-dimensional,
    #..the z-coordinate that is equal to zero is stored as well.
    #..Observe that the coordinates are stored contiguously for computational
    #..efficiency
    node_ids, node_coord, _ = gmsh.model.mesh.getNodes()
    nnodes = length(node_ids)
    #..sort the node coordinates by ID, such that Node one sits at row 1
    tosort = [node_ids node_coord[1:3:end] node_coord[2:3:end] node_coord[3:3:end]]; 
    sorted = sortslices(tosort , dims = 1);
    node_ids = sorted[:,1]
    xnode = sorted[:,2]
    ynode = sorted[:,3]
    znode = sorted[:,4]

    #..Finalize gmsh
    gmsh.finalize()
    
    return xnode,ynode,znode 
end

nodesFromGmsh (generic function with 1 method)

In [7]:
mesh = meshFromGmsh("cube_3D.msh");
xnode, ynode, znode = nodesFromGmsh("cube_3D.msh"); 
#display(mesh)

Info    : Reading 'cube_3D.msh'...
Info    : 27 entities
Info    : 14 nodes
Info    : 68 elements
Info    : Done reading 'cube_3D.msh'
Info    : Reading 'cube_3D.msh'...
Info    : 27 entities
Info    : 14 nodes
Info    : 68 elements
Info    : Done reading 'cube_3D.msh'


In [8]:
function genLocStiffMat(element::Element)
    p1 = element.p1; p2 = element.p2; p3 = element.p3; p4 = element.p4;
    e1 = element.e1; e2 = element.e2; e3 = element.e3; e4 = element.e4; 
    vol = element.vol 
    Iloc = SVector(e1, e1, e1, e1, e2, e2, e2, e2, e3, e3, e3, e3, e4, e4, e4, e4)
    Jloc = SVector(e1, e2, e3, e4, e1, e2, e3, e4, e1, e2, e3, e4, e1, e2, e3, e4)
    Xmat = SMatrix{4,4}(p1.x, p2.x, p3.x, p4.x, p1.y, p2.y, p3.y, p4.y, p1.z, p2.z, p3.z, p4.z, 1, 1, 1, 1) 
    rhs  = SMatrix{4,4}(1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1.) 
    Emat = MMatrix{4,4}(Xmat\rhs);
    Emat[4,:] .= 0.;  
    Amat = SMatrix{4,4}(vol*(transpose(Emat)*Emat));
    Aloc = [Amat[1,:] ; Amat[2,:] ; Amat[3,:] ; Amat[4,:]] 
    return Iloc, Jloc, Aloc
end

function genStiffMat(mesh::Mesh)
 
    #..recover number of elements  
    nelems     = mesh.nelements 
    dofPerElem = mesh.dofPerElement;
     
    #..preallocate the memory for local matrix contributions 
    Avalues = zeros(Float64,dofPerElem*nelems)
    I = zeros(Int64,length(Avalues))
    J = zeros(Int64,length(Avalues)) 

    for i = 1:nelems #..loop over number of elements..
        element          = mesh.Elements[i]
        Iloc, Jloc, Aloc = genLocStiffMat(element)
        irng             = mrange(dofPerElem*i-15, dofPerElem*i) 
        I[irng]          = Iloc 
        J[irng]          = Jloc 
        Avalues[irng]    = Aloc         
    end
    
    A = sparse(I,J,Avalues)
   
    return A; 
end

genStiffMat (generic function with 1 method)

In [9]:
Iloc, Jloc, Aloc = genLocStiffMat(mesh.Elements[1])
#@test Aloc == testAloc

([13, 13, 13, 13, 10, 10, 10, 10, 14, 14, 14, 14, 12, 12, 12, 12], [13, 10, 14, 12, 13, 10, 14, 12, 13, 10, 14, 12, 13, 10, 14, 12], [0.16666666666666666, -5.39691748081673e-18, -0.08333333333333333, -0.08333333333333333, -5.39691748081673e-18, 0.16666666666666674, -0.08333333333333334, -0.08333333333333337, -0.08333333333333333, -0.08333333333333334, 0.125, 0.041666666666666685, -0.08333333333333333, -0.08333333333333337, 0.041666666666666685, 0.125])

In [10]:
function genLocMassMat(element::Element)
    p1 = element.p1; p2 = element.p2; p3 = element.p3; p4 = element.p4;
    e1 = element.e1; e2 = element.e2; e3 = element.e3; e4 = element.e4; 
    vol = element.vol
    Iloc = SVector(e1, e1, e1, e1, e2, e2, e2, e2, e3, e3, e3, e3, e4, e4, e4, e4)
    Jloc = SVector(e1, e2, e3, e4, e1, e2, e3, e4, e1, e2, e3, e4, e1, e2, e3, e4)
    Mloc = SMatrix{3,3}(vol/4, 0., 0., 0., 0., vol/4, 0., 0., 0., 0., vol/4, 0., 0., 0., 0., vol/4) 
    return Iloc, Jloc, Mloc
end

function genMassMat(mesh::Mesh)
 
    #..recover number of elements  
    nelems     = mesh.nelements 
    dofPerElem = mesh.dofPerElement;
     
    #..preallocate the memory for local matrix contributions 
    Mvalues = zeros(Float64,dofPerElem*nelems)
    I = zeros(Int64,length(Mvalues))
    J = zeros(Int64,length(Mvalues)) 

    for i = 1:nelems #..loop over number of elements..
        element          = mesh.Elements[i]
        Iloc, Jloc, Mloc = genLocMassMat(element)
        irng             = mrange(dofPerElem*i-15, dofPerElem*i) 
        I[irng]          = Iloc 
        J[irng]          = Jloc 
        Mvalues[irng]    = Mloc         
    end
    
    M = sparse(I,J,Mvalues)
   
    return M; 
end

genMassMat (generic function with 1 method)

In [11]:
Iloc, Jloc, Mloc = genLocStiffMat(mesh.Elements[1])
#@test Aloc == testAloc

([13, 13, 13, 13, 10, 10, 10, 10, 14, 14, 14, 14, 12, 12, 12, 12], [13, 10, 14, 12, 13, 10, 14, 12, 13, 10, 14, 12, 13, 10, 14, 12], [0.16666666666666666, -5.39691748081673e-18, -0.08333333333333333, -0.08333333333333333, -5.39691748081673e-18, 0.16666666666666674, -0.08333333333333334, -0.08333333333333337, -0.08333333333333333, -0.08333333333333334, 0.125, 0.041666666666666685, -0.08333333333333333, -0.08333333333333337, 0.041666666666666685, 0.125])

In [12]:
mySourceFct(x,y,z) = x+y+z

function genLocVector(element::Element, sourceFct::Function)
    p1 = element.p1; p2 = element.p2; p3 = element.p3; p4 = element.p4;
    e1 = element.e1; e2 = element.e2; e3 = element.e3; e4 = element.e4; 
    vol = element.vol
    Iloc = SVector(e1, e2, e3, e4) 
    # use broadcast for the lines below instead 
    f1 = vol/4*sourceFct(p1.x,p1.y,p1.z)
    f2 = vol/4*sourceFct(p2.x,p2.y,p2.z)
    f3 = vol/4*sourceFct(p3.x,p3.y,p3.z)
    f4 = vol/4*sourceFct(p4.x,p4.y,p4.z)
    floc = SVector(f1, f2, f3, f4) 
    return Iloc, floc
end

function genVector(mesh, sourceFct::F) where F 
 
    #..recover number of elements  
    nelems  = mesh.nelements 
    nnodes = mesh.nnodes 
     
    #..preallocate the memory for local matrix contributions 
    f = zeros(Float64,nnodes)

    for i = 1:nelems #..loop over number of elements..
        element::Element = mesh.Elements[i]
        Iloc, floc = genLocVector(element,sourceFct)
        f[Iloc] += floc 
    end
       
    return f; 
end

genVector (generic function with 1 method)

In [13]:
function handleBoundary!(mesh,A,f)
    bndNodeIds = mesh.bndNodeIds; 
    #..handle essential boundary conditions 
    A[bndNodeIds,:] .= 0;
    A[bndNodeIds,bndNodeIds] = Diagonal(ones(size(bndNodeIds)))
    f[bndNodeIds] .= 1/(3*(pi)^2);
    return A, f  
end

handleBoundary! (generic function with 1 method)

In [15]:
mesh = meshFromGmsh("cube_3D.msh"); A = genStiffMat(mesh); f = genVector(mesh,mySourceFct);
A,f = handleBoundary!(mesh,A,f);

Info    : Reading 'cube_3D.msh'...
Info    : 27 entities
Info    : 14 nodes
Info    : 68 elements
Info    : Done reading 'cube_3D.msh'


In [16]:
function generateSolution(mesh,A,f)

    A, f = handleBoundary!(mesh,A,f)
    u = A\f 
    return u 
end

generateSolution (generic function with 1 method)

In [17]:
#mesh = meshFromGmsh("square-10.msh");   
#A = genStiffMat(mesh); f= generateVector(mesh,mySourceFct); # <= force recompilation
u = generateSolution(mesh,A,f)

14-element Vector{Float64}:
 0.03377372788077926
 0.03377372788077926
 0.03377372788077926
 0.03377372788077926
 0.03377372788077926
 0.03377372788077926
 0.03377372788077926
 0.03377372788077926
 0.03377372788077926
 0.03377372788077926
 0.03377372788077926
 0.03377372788077926
 0.03377372788077926
 0.03377372788077926

In [18]:
#GR.trisurf(xnode,ynode,znode,u)

In [19]:
# write 3D mesh data to file 
# 3D mesh
function mesh_data_VTK(mesh, nodes,u)

    # Create cells (all hexahedrons in this case) and cell data.
    celltype = VTKCellTypes.VTK_TETRAHEDRON
    cells = MeshCell{VTKCellType, SVector{4, Int}}[]

    for i = 1:nelements #..loop over number of elements..
        element = mesh.Elements[i]
        inds = MVector{4, Int}(undef)
        inds[1] = u[element.e1]
        inds[2] = u[element.e2]
        inds[3] = u[element.e3]
        inds[4] = u[element.e4]

        # Define cell.
        c = MeshCell(celltype, SVector(inds))

        push!(cells, c)
    end

    return pts, cells

end

points, cells = mesh_data_VTK(mesh, nodes, u)
vtk_grid("filename", points, cells) do vtk
    vtk["solution"] = u 
end

LoadError: UndefVarError: nodes not defined

In [ ]:
#mesh_data_VTK(mesh, nodes,u)